In [28]:
# imports
import os
import pickle
import time
import pandas as pd

In [2]:
# changing directory
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [22]:
# get the data
continents = pd.read_csv('data/continents.csv')
country_confirmed = pd.read_csv("data/country_confirmed.csv")
country_deaths = pd.read_csv("data/country_deaths.csv")
country_recovered = pd.read_csv("data/country_recovered.csv")
countries = pd.read_csv("data/countries.csv")
global_ts = pd.read_csv("data/global_ts.csv")
vac_global = pd.read_csv("data/vac_global.csv")
vac_country = pd.read_csv("data/vac_country.csv")
with open("data/global_update.pkl", "rb") as file:
    global_update = pickle.load(file)


In [23]:
continents.index = continents['continent']
continents.drop(columns=['continent'], inplace=True)

In [26]:
# continents.loc["North America"]
continents

,lat,long,confirmed_cumulative,confirmed_new,deaths_cumulative,deaths_new,recovered_cumulative,recovered_new,active,critical,updated
continent,,,,,,,,,,,
North America,31.676827,-146.470747,48995944,7504,1009253,280,38170917,11151,9815774,33093,1630912316288
Asia,23.702727,62.375064,71353320,29794,1054534,427,66679631,44324,3619155,41043,1630912316289
South America,-15.655156,-100.748423,37083686,171,1135465,5,34916232,643,1031989,14837,1630912316290
Europe,25.771324,-35.601226,55970233,1906,1180867,30,51052700,7920,3736666,11914,1630912316291
Africa,1.738387,-16.309464,7984085,0,199688,0,7132291,0,652106,4468,1630912316292
Australia-Oceania,-8.659916,91.146985,173518,1557,2259,5,120066,38,51193,233,1630912316292
